# Breadboard grid extraction

First detect all breadboard holes and then use them to predict the grids on a breadboard so that hidden breadboard holes can be predicted

In [1]:
import cv2
import numpy as np
from sklearn.linear_model import RANSACRegressor
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

In [2]:


# Set up SimpleBlobDetector parameters
params = cv2.SimpleBlobDetector_Params()
params.filterByArea = True
params.minArea = 50
#params.maxArea = 100
params.filterByColor = True
params.blobColor = 0  # 0 for dark blobs, 255 for light blobs
params.filterByCircularity = True
params.minCircularity = 0.7
params.filterByConvexity = False
params.filterByInertia = False

# Create detector
detector = cv2.SimpleBlobDetector_create(params)

In [ ]:
# Load image in grayscale
image = cv2.imread("bb3.jpg", cv2.IMREAD_GRAYSCALE)
colour_image = cv2.imread("bb2.jpg")

# Downsample to 720p if larger
max_height, max_width = 720, 1280
h, w = image.shape
if h > max_height or w > max_width:
    scale = min(max_width / w, max_height / h)
    new_size = (int(w * scale), int(h * scale))
    image = cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)

# Assume colour_image is your input
gray = cv2.cvtColor(colour_image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(blurred, 50, 150)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter and draw contours
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    if w > 5 and h > 30:  # Example thresholds
        cv2.rectangle(colour_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imshow("Detected Legs", colour_image)
cv2.waitKey(0)

Grid Generation stuff

In [130]:
# === 1. Extract Keypoint Coordinates ===
def get_keypoint_coords(keypoints):
    return np.array([kp.pt for kp in keypoints])

# === 2. Cluster Rows (along Y) ===
def cluster_rows(pts, eps=3):
    clustering = DBSCAN(eps=eps, min_samples=4).fit(pts[:, 1].reshape(-1, 1))
    return clustering.labels_

# === 3. Cluster Columns (along X) ===
def cluster_columns(pts, eps=3):
    clustering = DBSCAN(eps=eps, min_samples=4).fit(pts[:, 0].reshape(-1, 1))
    return clustering.labels_

# === 4. Fit Row Lines using RANSAC (y = mx + b) ===
def fit_horizontal_lines(pts, labels):
    lines = []
    for label in np.unique(labels):
        if label == -1:
            continue
        row_pts = pts[labels == label]
        X = row_pts[:, 0].reshape(-1, 1)  # x
        y = row_pts[:, 1]                # y

        model = RANSACRegressor().fit(X, y)
        slope = model.estimator_.coef_[0]
        intercept = model.estimator_.intercept_
        lines.append((slope, intercept))
    return lines

# === 5. Fit Column Lines using RANSAC (x = m*y + b) ===
def fit_vertical_lines(pts, labels, min_points=3):
    lines = []
    for label in np.unique(labels):
        if label == -1:
            continue
        col_pts = pts[labels == label]
        if len(col_pts) < min_points:
            continue
        Y = col_pts[:, 1].reshape(-1, 1)  # y
        X = col_pts[:, 0]                # x

        model = RANSACRegressor().fit(Y, X)
        slope = model.estimator_.coef_[0]
        intercept = model.estimator_.intercept_
        lines.append((slope, intercept))
    return lines

# === 6. Generate Line Points ===
def line_points(slope, intercept, x_range):
    x = np.linspace(*x_range, num=2)
    y = slope * x + intercept
    return np.column_stack([x, y])

def vertical_line_points(slope, intercept, y_range):
    y = np.linspace(*y_range, num=2)
    x = slope * y + intercept
    return np.column_stack([x, y])

# === 7. Drawing Functions ===
def draw_lines(image, lines, x_range, color):
    for slope, intercept in lines:
        pts = line_points(slope, intercept, x_range).astype(int)
        cv2.line(image, tuple(pts[0]), tuple(pts[1]), color, 1)

def draw_vertical_lines(image, lines, y_range, color):
    for slope, intercept in lines:
        pts = vertical_line_points(slope, intercept, y_range).astype(int)
        cv2.line(image, tuple(pts[0]), tuple(pts[1]), color, 1)

def find_intersections(row_lines, col_lines):
    intersections = []
    for m_row, b_row in row_lines:
        for m_col, b_col in col_lines:
            # row: y = m_row * x + b_row
            # col: x = m_col * y + b_col
            # Solve for x and y
            # Substitute x from col into row:
            # y = m_row * (m_col * y + b_col) + b_row
            # y - m_row * m_col * y = m_row * b_col + b_row
            # y * (1 - m_row * m_col) = m_row * b_col + b_row
            denom = 1 - m_row * m_col
            if abs(denom) < 1e-6:
                continue  # Lines are nearly parallel, skip
            y = (m_row * b_col + b_row) / denom
            x = m_col * y + b_col
            intersections.append((x, y))
    return np.array(intersections)

def print_column_clusters(pts, col_labels):
    for label in np.unique(col_labels):
        if label == -1:
            continue
        col_pts = pts[col_labels == label]
        print(f"Column cluster {label}:")
        for pt in col_pts:
            print(f"    {pt}")

# === 8. Full Grid Generation Pipeline ===
def build_grid_from_keypoints(image, keypoints, row_eps=5, col_eps=3):
    img_copy = image.copy()
    pts = get_keypoint_coords(keypoints)

    # Cluster and fit horizontal lines
    row_labels = cluster_rows(pts, eps=row_eps)
    row_lines = fit_horizontal_lines(pts, row_labels)

    # Cluster and fit vertical lines
    col_labels = cluster_columns(pts, eps=col_eps)
    col_lines = fit_vertical_lines(pts, col_labels)

    # Print all points for each column cluster
    print_column_clusters(pts, col_labels)

    # Draw horizontal lines (red)
    draw_lines(img_copy, row_lines, x_range=(0, image.shape[1]), color=(0, 0, 255))

    # Draw vertical lines (green)
    draw_vertical_lines(img_copy, col_lines, y_range=(0, image.shape[0]), color=(0, 255, 0))

    return img_copy, row_lines, col_lines

In [131]:
# Detect blobs
keypoints = detector.detect(image)

# Draw detected keypoints
im_with_keypoints = cv2.drawKeypoints(
    image, keypoints, np.array([]), (0, 255, 0),
    cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)

# Build grid
grid_img, rows, cols = build_grid_from_keypoints(im_with_keypoints, keypoints)

# Find intersections
intersections = find_intersections(rows, cols)

# Draw intersection points
for x, y in intersections:
    cv2.circle(grid_img, (int(round(x)), int(round(y))), 3, (0, 0, 255), -1)

cv2.imshow("Grid", grid_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Column cluster 0:
    [1032.87316895  434.4246521 ]
    [1032.51574707  416.45095825]
    [1032.44238281  343.177948  ]
    [1032.28088379  288.82873535]
    [1033.08190918  214.57751465]
    [1032.9206543   196.00953674]
    [1033.60461426  564.41809082]
    [1033.41552734  489.19213867]
    [1032.98046875  470.92211914]
    [1032.68273926  361.32333374]
    [1032.47460938  325.13256836]
    [1032.42333984  306.9630127 ]
    [1033.85522461  583.21588135]
Column cluster 1:
    [1049.94042969  434.16516113]
    [1050.45227051  416.44787598]
    [1050.36767578  343.11328125]
    [1050.30664062  324.96112061]
    [1050.30163574  306.81375122]
    [1050.18444824  288.69726562]
    [1051.07128906  214.61372375]
    [1051.8548584   582.95111084]
    [1051.38781738  564.26525879]
    [1050.4375      361.21710205]
    [1050.84008789  196.12815857]
    [1051.44873047  489.1920166 ]
    [1050.89599609  470.94598389]
Column cluster 2:
    [1014.69580078  434.24728394]
    [1014.48028564  416.5083